In [177]:
from metrics import *
from utils import *
from data_utils import *
from plotting import *
import lstm
import pdb
import pandas as pd
import numpy as np
import keras
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import plotly
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)

#DATAFILE = '../analyser/rawData/2018-01-15_ETHBTC_1m_6_mon_slice_last_50k.csv'
#DATAFILE = '../analyser/rawData/2018-01-15_ETHBTC_1m_6_mon_.csv'
DATAFILE = '../analyser/rawData/2018-01-15_ETHBTC_1h_4_mon_.csv'
#DATAFILE = '../analyser/rawData/2018-02-13_ETHBTC_5m_2_mon_.csv'
#DATAFILE = '../analyser/rawData/test-sample-mixed.csv'
#DATAFILE = '../analyser/rawData/test-sample-flat.csv'
#DATAFILE = '../analyser/rawData/test-sample-falling.csv'
#DATAFILE = '../analyser/rawData/2018-02-13_ETHBTC_1m_2_mon_.csv'
#DATAFILE = 'test.csv'
COLUMN_NAME = 'close'

#DATAFILE = 'shampoo-data.csv'
#COLUMN_NAME = 'Sales'
#DATAFILE = 'sinwave.csv'
#COLUMN_NAME = 'sin'

In [178]:
# Load data
#DATAFILE = 'shampoo-data.csv'
#COLUMN_NAME = 'Sales'
data = pd.read_csv(DATAFILE)
prices = data[COLUMN_NAME]

pricesLine = Scatter(
            x=prices.index,
            y=prices,
            name='prices'
            )
iplot([pricesLine], config={ "scrollZoom": True })




In [179]:
# Detrend
diffPr = np.log(prices).diff().dropna()
logDiffLine = Scatter(
            x=diffPr.index,
            y=diffPr,
            name='log diff'
            )
iplot([logDiffLine], config={ "scrollZoom": True })



In [180]:
# Split training set and test set
dtrain, dtest = splitDataset(0.8, 0.2, diffPr)
xtrain = dtrain.shift().iloc[1:].values
xtrain = xtrain.reshape(xtrain.shape[0], 1, 1)

ytrain = dtrain.iloc[1:].values
ytrain = ytrain.reshape(ytrain.shape[0], 1)

xtest = dtest.shift().iloc[1:].values
xtest = xtest.reshape(xtest.shape[0], 1, 1)

ytest = dtest.iloc[1:].values
ytest = ytest.reshape(xtest.shape[0], 1)

print('Training examples: {}'.format(xtrain.shape[0]))
print('Test examples: {}'.format(xtest.shape[0]))


Training examples: 2090
Test examples: 521


In [184]:
# Train the model
EPOCHS = 10
model = Sequential()
model.add(LSTM(128,
    batch_input_shape=(1, xtrain.shape[1], xtrain.shape[2]),
    stateful=True
    ))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss="mean_squared_error", optimizer="adam")
reduce_lr = keras.callbacks.ReduceLROnPlateau(
            monitor='loss',
            factor=0.9,
            patience=10,
            min_lr=0.000001,
            verbose=1)
print("Start training for {} epochs".format(EPOCHS))
for i in range(EPOCHS):
    model.fit(
        xtrain,
        ytrain,
        epochs=1,
        batch_size=1,
        verbose=1,
        shuffle=False,
        callbacks=[reduce_lr]
    )
    model.reset_states()
    print("{}/{} epoch complete".format(i + 1, EPOCHS))


Start training for 5000 epochs
Epoch 1/1
2090/2090 [==============================] - 10s 5ms/step - loss: 2.6911e-04
1/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.1320e-04
2/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0307e-04
3/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0135e-04
4/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0116e-04
5/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0128e-04
6/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0168e-04
7/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0160e-04
8/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 10s 5ms/step - loss: 2.0028e-04
9/5000 epoch comp

2090/2090 [==============================] - 8s 4ms/step - loss: 2.0133e-04A: 0s - loss: 2.0161e-0
75/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 7s 4ms/step - loss: 2.0112e-04
76/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0094e-04
77/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0068e-04
78/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0172e-04
79/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 7s 4ms/step - loss: 2.0085e-04
80/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 7s 4ms/step - loss: 2.0088e-04
81/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0244e-04
82/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0064e-04
83/5000 epoch complete
Epoch 

2090/2090 [==============================] - 10s 5ms/step - loss: 2.0130e-04
149/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 9s 4ms/step - loss: 2.0173e-04
150/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0112e-04
151/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 9s 4ms/step - loss: 2.0142e-04
152/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 10s 5ms/step - loss: 2.0115e-04
153/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 14s 6ms/step - loss: 2.0088e-04
154/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 9s 4ms/step - loss: 2.0100e-04
155/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 9s 4ms/step - loss: 2.0103e-04
156/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0116e-04
157/5000 epoch complete
Epoch 1/1
2090/20

2090/2090 [==============================] - 9s 4ms/step - loss: 2.0134e-04
223/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 9s 4ms/step - loss: 2.0124e-04
224/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0096e-04
225/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0133e-04
226/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0126e-04
227/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 9s 4ms/step - loss: 2.0050e-04
228/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 9s 4ms/step - loss: 2.0137e-04
229/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 9s 4ms/step - loss: 2.0160e-04
230/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 9s 4ms/step - loss: 2.0140e-04
231/5000 epoch complete
Epoch 1/1
2090/2090 

2090/2090 [==============================] - 7s 3ms/step - loss: 2.0125e-04
297/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 7s 3ms/step - loss: 2.0156e-04A:
298/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 7s 3ms/step - loss: 2.0100e-04
299/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 7s 3ms/step - loss: 2.0087e-04
300/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 7s 3ms/step - loss: 2.0097e-04
301/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 7s 3ms/step - loss: 2.0131e-04
302/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 7s 3ms/step - loss: 2.0140e-04
303/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 7s 3ms/step - loss: 2.0078e-04
304/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 7s 3ms/step - loss: 2.0089e-04
305/5000 epoch complete
Epoch 1/1
2090/209

...

2090/2090 [==============================] - 8s 4ms/step - loss: 2.0111e-04A: 0s - loss:
4649/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0138e-04A: 0s - loss: 2.0
4650/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0140e-04A:  - E
4651/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0155e-04
4652/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0071e-04
4653/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0083e-04
4654/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0145e-04A: 0s - loss: 2.
4655/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0126e-04
4656/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - 

2090/2090 [==============================] - 8s 4ms/step - loss: 2.0140e-04
4720/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0061e-04
4721/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0096e-04
4722/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0042e-04A: 0s - loss: 2.038
4723/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0159e-04
4724/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0109e-04 ETA: 0s - 
4725/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0120e-04
4726/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0086e-04
4727/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0092e-04
4728/5

2090/2090 [==============================] - 8s 4ms/step - loss: 2.0036e-04A: 0s - loss: 2.0341e
4792/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0168e-04
4793/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0088e-04
4794/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0088e-04
4795/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0106e-04
4796/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0116e-04
4797/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0144e-04
4798/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0148e-04A:
4799/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0103e-04
4800/5000 epo

2090/2090 [==============================] - 8s 4ms/step - loss: 2.0123e-04
4863/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0048e-04
4864/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0241e-04
4865/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0104e-04
4866/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0159e-04
4867/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0101e-04
4868/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0115e-04A: 1s - loss:  - ETA
4869/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0087e-04A: 2s - loss: 1.3310 - ETA: 2
4870/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss

2090/2090 [==============================] - 8s 4ms/step - loss: 2.0082e-04
4933/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0065e-04A: 1s - loss: 1.6862 - ETA: 1s - loss: 1.79 - ETA: 0s 
4934/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0104e-04
4935/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0167e-04
4936/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0026e-04
4937/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0192e-04
4938/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0141e-04A: 0s - loss: 2.0182e-0
4939/5000 epoch complete
Epoch 1/1
2090/2090 [==============================] - 8s 4ms/step - loss: 2.0099e-04- ETA: 0s - loss
4940/5000 epoch complete
Epoch 1/1
2090/2090 [========

In [195]:
# Evaluate
# Preseed with train data
PRESEED_AMOUNT = 500
preseedX = xtrain[-PRESEED_AMOUNT:]
for i in range(preseedX.shape[0]):
    model.predict(preseedX[i].reshape(1,1,1))

# Predict the values from test
pred = np.array([[0]])
for i in range(xtest.shape[0]):
    pr = model.predict(xtest[i].reshape(1,1,1))
    pred = np.concatenate((pred, pr))

pred = pred[1:]
model.reset_states()

In [188]:
# Plot as it is
predLine = Scatter(
            x=np.arange(pred.shape[0]),
            y=pred.flatten(),
            name='predictions'
            )
actualLine = Scatter(
            x=np.arange(ytest.shape[0]),
            y=ytest.flatten(),
            name='actual'
            )
iplot([predLine, actualLine], config={ "scrollZoom": True })



In [152]:
firstTestPrice = prices[dtest.index[0]]
predPrice = np.exp(pred.cumsum() + np.log(firstTestPrice))
actualPrice = np.exp(ytest.cumsum() + np.log(firstTestPrice))
